In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import pillow_heif
from PIL.ExifTags import TAGS
import piexif
from pillow_heif import register_heif_opener

os.chdir("/home/michael/AmbientLight/")

from src.utils import extract_specific_metadata, create_hdr_from_images





In [2]:
[1, 2]

[1, 2]

Final thing: Ambient Light hitting sclera and entire image

1. Run debevik and malik on diff exposure images --> get HDR image (floats of 10^-5 to 10^4 ish) / relative radiance


1. Function to calculate ambient light from a given image: (mean pixel value/calculatedExposure)

it estaimtes camera response function too?

add alignment algorithm

!@! radiance of pupil/sclera is proportional to irradiance given that reflectance is ~constant
-----

get camera response function

does debevik and malik help at all? maybe only if you have the reference...



metrics... 
- calculated exposure with ev = 0
-  currentBrightness = UIScreen.main.brightness


take picture with widest angle?
front and back
use raws?
average over time (median of pictures)
make the person move around?

clip highlights above a threshold? ... 5-95 or something of pixels only

RAW or “manual” mode with zero “smart HDR,” minimal or no local tone mapping, and locked white balance if possible.

do log average instead of average


!@! - have them do a long exposure shot while sweeping like a panorama?
- can also just do multiple shots and average tho...

gaussian blur images to simulate diffusion? a median blur?

darkest 20-30% of pixels of pupil?

--------------------------
Potential Algorithm:

Person spins 360˚, ~10 photos are taken


Could do an algorithm as simple as taking many shots on widest camera or whatever with auto exposure on... just look at calculated_exposure

Lux≈2.5×2 
EV 
100
​

Lux ~= 2.5 * 10^EV100

EV100 = log2(Aperture^2/shutter speed)

EV100 = exposure value at 100 iso



Lux ~= 2.5 * aperture^2 * 100 / shutter speed / iso || assuming mean expsoure

adjusted: 
Lux ~= 2.5 * aperture^2 * 100 / shutter speed / iso * (mean pixel value/128)

RAW = 12-14 bits of dynamic range


my thing before: exposure_time * iso / (f_number**2)


skip HDR thing... just do mutliple pictures while moving around and do lux estimation thing
do wide angle 
allow autofocus

E = Lgray / calculatedExposure / R

Lgray = radiance value of gray thing from HDR map
R = reflectance


In [5]:

image_paths = [
    "test_photos_heic/face-2.HEIC",
    "test_photos_heic/face0.HEIC",
    "test_photos_heic/face+2.HEIC"
]

image_paths = [
    "test_photos_heic/table-2.HEIC",
    "test_photos_heic/table0.HEIC",
    "test_photos_heic/table+2.HEIC"
]

metadata_list = extract_specific_metadata(image_paths)

for metadata in metadata_list:
    print(f"Metadata for {metadata['filename']}:")
    for key in ["ExposureTime", "FNumber", "ISOSpeedRatings", "BrightnessValue", "ExposureBiasValue", "calculatedExposure"]:
        value = metadata.get(key)
        if value is not None:
            print(f"  {key}: {value}")
        else:
            print(f"  {key}: None")
    print("\n")


Metadata for test_photos_heic/table-2.HEIC:
  ExposureTime: 0.008333333333333333
  FNumber: 2.2
  ISOSpeedRatings: 125
  BrightnessValue: 2.348783647617677
  ExposureBiasValue: -2.0
  calculatedExposure: 0.21522038567493113


Metadata for test_photos_heic/table0.HEIC:
  ExposureTime: 0.016666666666666666
  FNumber: 1.7799999713880652
  ISOSpeedRatings: 250
  BrightnessValue: 1.9762883948078653
  ExposureBiasValue: 0.0
  calculatedExposure: 1.3150696883656343


Metadata for test_photos_heic/table+2.HEIC:
  ExposureTime: 0.016666666666666666
  FNumber: 1.7799999713880652
  ISOSpeedRatings: 800
  BrightnessValue: 1.9325907590759075
  ExposureBiasValue: 2.0
  calculatedExposure: 4.20822300277003




In [6]:
metadata_list

[{'filename': 'test_photos_heic/table-2.HEIC',
  'ExposureTime': 0.008333333333333333,
  'FNumber': 2.2,
  'ISOSpeedRatings': 125,
  'BrightnessValue': 2.348783647617677,
  'ExposureBiasValue': -2.0,
  'calculatedExposure': 0.21522038567493113,
  'lux_unadjusted': 1161.6000000000001,
  'lux_adjusted': 78.43301250535269,
  'mean_pixel_value': 37.59570944021899},
 {'filename': 'test_photos_heic/table0.HEIC',
  'ExposureTime': 0.016666666666666666,
  'FNumber': 1.7799999713880652,
  'ISOSpeedRatings': 250,
  'BrightnessValue': 1.9762883948078653,
  'ExposureBiasValue': 0.0,
  'calculatedExposure': 1.3150696883656343,
  'lux_unadjusted': 190.10399388849078,
  'lux_adjusted': 137.8318797502401,
  'mean_pixel_value': 110.59526469901687},
 {'filename': 'test_photos_heic/table+2.HEIC',
  'ExposureTime': 0.016666666666666666,
  'FNumber': 1.7799999713880652,
  'ISOSpeedRatings': 800,
  'BrightnessValue': 1.9325907590759075,
  'ExposureBiasValue': 2.0,
  'calculatedExposure': 4.20822300277003,
 

In [3]:
create_hdr_from_images(image_paths)

Loaded test_photos_heic/face-2.HEIC with dimensions: (3088, 2316, 3)
Loaded test_photos_heic/face0.HEIC with dimensions: (3088, 2316, 3)
Loaded test_photos_heic/face+2.HEIC with dimensions: (3088, 2316, 3)
All images have the same dimensions. No resizing needed.
Calibrating and merging with Debevec...
Scaling HDR image to 8-bit range...
HDR Image - Min Radiance: 0.003833844792097807, Max Radiance: 63.14649200439453
Saved Debevec HDR to: hdr_debevec.jpg


In [ ]:
# List of exposure images and their exposure times
image_paths = ["test_photos_heic/table_-2.HEIC", "test_photos_heic/table_0.HEIC", "test_photos_heic/table_+2.HEIC"]

exposure_times = np.array([1/30.0, 1/60.0, 1/125.0], dtype=np.float32)  # Corresponding exposure times

# Load images
images = [cv2.imread(img) for img in image_paths]
